## Birds Calls Preprocessing

## PIP Installs

In [14]:
# PIPs
# !pip install pydub
# !pip install timm==0.4.5
# !pip install audiomentations
# !pip install torch_audiomentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Drive Mount

In [15]:
# mount gdrive

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Imports

In [16]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import pickle
import json

from pathlib import Path

sys.path.append('/content/gdrive/My Drive/DeepLearning/project/')
sys.path.append('/content/gdrive/My Drive/DeepLearning/project/SSAST/src')
!cd '/content/gdrive/My Drive/DeepLearning/project/SSAST/src'

# cd gdrive/My Drive/DeepLearning/project/
import wav_processing as wp
import project_utils
import birds_run
import birds_ssast_data_setting as birds_ds
from data_validator import Validator

## File Path

In [17]:
working_path = '/content/gdrive/My Drive/DeepLearning'
# excel_path = '/content/gdrive/My Drive/DeepLearning/bird_records/audiomoth_general.xlsx'
excel_path = '/content/gdrive/My Drive/DeepLearning/bird_records/audiomoth_general 2022 09.xlsx'


records_path = '/content/gdrive/My Drive/DeepLearning/bird_records/'
record_list_path = '/content/gdrive/My Drive/DeepLearning/project/records_list.pickle'

labeld_data_path = '/content/gdrive/My Drive/DeepLearning/project/labeled_data/'

ssast_src_path = '/content/gdrive/My Drive/DeepLearning/project/SSAST/src'
saved_models = '/content/gdrive/My Drive/DeepLearning/project/experiments/00 Saved Models/'

## Examine Raw Data

In [19]:
mf = wp.MarkedFiles(excel_path)
file_list = mf.get_file_list(records_path)
len(file_list)

350

In [20]:
mf.count_labels()
mf.labels_count.keys()

dict_keys(['CommonMyna', 'HouseSparrow', 'EurasianCollaredDove', 'LesserWhitethroat', 'Pycnonotus', 'GracefulPrinia', 'HoodedCrow', 'White-ThroatedKingfisher', 'Pycnonotus/CommonChaffinch', 'CommonChaffinch', 'PalestineSunbird', 'Unknown', 'Spur-WingedLapwing', "WesternBonelli'SWarbler", 'EurasianBlackcap', 'Rose-RingedParakeet', 'EurasianBlackcap/LesserWhitethroat', 'GreatTit', 'EurasianJay', 'EuropeanGreenfinch', 'CommonSwift', 'PalestineSunbird?', 'PalestineSunbird/LesserWhitethroat', 'Seagull?', 'CommonMyna/PalestineSunbird?', 'EuropeanGoldfinch', 'GracefulPrinia/HouseSparrow', 'Upupa', 'HoddedCrow', 'CommonMyna/EuropeanGoldfinch?', 'BarnSwallow?', 'BarnSwallow', 'GracefulPrinia?', 'BarnSwallow/PalestineSunbird', 'Upupa?', 'Rose-RingedParakeet/MonkParakeet?', 'HouseSparrow?', 'MonkParakeet', 'MonkParakeet/CommonMyna', 'CommonMyna/MonkParakeet', 'CommonChiffchaff/PalestineSunbird', 'Goose/Seagull?', 'Seagull?/Mallard', 'EurasianStone-Curlew', 'CommonChaffinch/GreatTit', 'GreatTit?',

In [ ]:
count_th = 10

labels_in = 0
labels_out = 0

mf.count_labels()
for label in mf.labels_count.keys():
  if mf.labels_count[label] > count_th:
    print(f'{label}={mf.labels_count[label]}')
    labels_in += 1
  else:
    labels_out += 1

print()
print(f'labels_in={labels_in}')
print(f'labels_out={labels_out}')

# mf.labels_count

CommonMyna=2137
EurasianCollaredDove=633
LesserWhitethroat=354
Pycnonotus=1555
GracefulPrinia=895
HoodedCrow=1075
White-ThroatedKingfisher=267
CommonChaffinch=34
PalestineSunbird=449
Spur-WingedLapwing=61
EurasianBlackcap=64
Rose-RingedParakeet=363
EurasianBlackcap/LesserWhitethroat=16
GreatTit=1371
EuropeanGreenfinch=54
CommonSwift=89
PalestineSunbird?=41
EuropeanGoldfinch=61
Upupa=40
BarnSwallow?=16
BarnSwallow=35
GracefulPrinia?=35
Rose-RingedParakeet/MonkParakeet?=22
HouseSparrow?=13
MonkParakeet=2084
EgyptianGoose=140
EgyptianGoose?=14
LaughingDove=237
MonkParakeet/Rose-RingedParakeet=12
EuropeanBee-Eater=31
CommonNightingale=12
CommonKingfisher=71
CommonBlackbird=101
EuropeanRobin=158
CommonRedstart=52

labels_in=35
labels_out=42


## Data Setting Full Cycle

In [21]:
birds_ds.prepare_dataset(excel_path, orig_records_path=records_path, dst_dir_path=labeld_data_path, partial=None, low_count_label_filter=1000, string_filter_list=['?', '/'])

Loading excel data from /content/gdrive/My Drive/DeepLearning/bird_records/audiomoth_general 2022 09.xlsx
Generate labels csv /content/gdrive/My Drive/DeepLearning/project/labeled_data/birdcalls_class_labels_indices.csv

Found 350 different files on marking excel
Splitting original records from /content/gdrive/My Drive/DeepLearning/bird_records/ to 1sec slices 
Arrange files in train/test/validate directories on /content/gdrive/My Drive/DeepLearning/project/labeled_data/
Now processing train part
Length 252


100%|██████████| 252/252 [01:40<00:00,  2.52it/s]


Now processing validate part
Length 28


100%|██████████| 28/28 [00:11<00:00,  2.42it/s]


Now processing test part
Length 70


100%|██████████| 70/70 [00:27<00:00,  2.59it/s]



Generate json files for SSAST in {dst_dir_path}
Generating with following blacklist:
['HouseSparrow', 'EurasianCollaredDove', 'LesserWhitethroat', 'GracefulPrinia', 'White-ThroatedKingfisher', 'Pycnonotus/CommonChaffinch', 'CommonChaffinch', 'PalestineSunbird', 'Unknown', 'Spur-WingedLapwing', "WesternBonelli'SWarbler", 'EurasianBlackcap', 'Rose-RingedParakeet', 'EurasianBlackcap/LesserWhitethroat', 'EurasianJay', 'EuropeanGreenfinch', 'CommonSwift', 'PalestineSunbird?', 'PalestineSunbird/LesserWhitethroat', 'Seagull?', 'CommonMyna/PalestineSunbird?', 'EuropeanGoldfinch', 'GracefulPrinia/HouseSparrow', 'Upupa', 'HoddedCrow', 'CommonMyna/EuropeanGoldfinch?', 'BarnSwallow?', 'BarnSwallow', 'GracefulPrinia?', 'BarnSwallow/PalestineSunbird', 'Upupa?', 'Rose-RingedParakeet/MonkParakeet?', 'HouseSparrow?', 'MonkParakeet/CommonMyna', 'CommonMyna/MonkParakeet', 'CommonChiffchaff/PalestineSunbird', 'Goose/Seagull?', 'Seagull?/Mallard', 'EurasianStone-Curlew', 'CommonChaffinch/GreatTit', 'Great

## Only jsons - With Bugs - Don't Use

In [ ]:
# Don't use

# dst_dir_path=labeld_data_path
# low_count_label_filter = 10

# mf.generate_json(json_dst_dir=dst_dir_path,
#                      part_list_path=Path(dst_dir_path, 'part_lists.json'),
#                      wav_record_path=dst_dir_path,
#                      partial=None,
#                      low_count_label_filter=None) # low_count_label_filter)

Processing train files
Processing validate files
Processing test files


## Off-line Augmentation

In [ ]:
import apply_pre_train_augmentation as apply_pre_train_aug

#json_train_file, target_path

train_data_json = 'birdcalls_train_data_low_count_filter_10.json'

target_path = Path(labeld_data_path, 'labeled_aug_data')
json_train_file = Path(labeld_data_path, train_data_json)
json_dst_dir = target_path

pre_train_aug = apply_pre_train_aug.PreTrainAugmentFiles(json_train_file, target_path)

pre_train_aug.create_augmented_files(json_dst_dir)

## Validation

In [ ]:
# Validation
labels_csv_path = Path(labeld_data_path, 'birdcalls_class_labels_indices.csv')
part_list_path = Path(labeld_data_path, 'part_lists.json')
labeled_path_dict = {
    'train': Path(labeld_data_path, 'birdcalls_train_data.json'),
    'validate': Path(labeld_data_path, 'birdcalls_validate_data.json'),
    'test': Path(labeld_data_path, 'birdcalls_test_data.json')
}

birds_ds.validate_data(excel_path, labels_csv_path, part_list_path, labeled_path_dict)
# v = Validator(excel_path, labels_csv_path, part_list_path, labeled_path_dict)
# v.validate_labels_on_jsons(show=True)

Streaming output truncated to the last 5000 lines.
invalid record /content/gdrive/My Drive/DeepLearning/project/labeled_data/train/2_20200325_060900_17.WAV
record labels are ['/m/birdc59']
invalid record /content/gdrive/My Drive/DeepLearning/project/labeled_data/train/2_20200325_060900_18.WAV
record labels are ['/m/birdc19', '/m/birdc59']
invalid record /content/gdrive/My Drive/DeepLearning/project/labeled_data/train/2_20200325_060900_19.WAV
record labels are ['/m/birdc59']
invalid record /content/gdrive/My Drive/DeepLearning/project/labeled_data/train/2_20200325_060900_20.WAV
record labels are ['/m/birdc59']
invalid record /content/gdrive/My Drive/DeepLearning/project/labeled_data/train/2_20200325_060900_21.WAV
record labels are ['/m/birdc19', '/m/birdc59']
invalid record /content/gdrive/My Drive/DeepLearning/project/labeled_data/train/2_20200325_060900_22.WAV
record labels are ['/m/birdc19', '/m/birdc59']
invalid record /content/gdrive/My Drive/DeepLearning/project/labeled_data/train

KeyboardInterrupt: ignored

In [ ]:
v.invalid_json_records[:10]

In [ ]:
labeled_path_dict = {
    'train': Path(labeld_data_path, 'birdcalls_train_data.json'),
    'validate': Path(labeld_data_path, 'birdcalls_validate_data.json'),
    'test': Path(labeld_data_path, 'birdcalls_test_data.json')
}


In [ ]:
with open(labeled_path_dict['train'], 'r') as f:
    file_data_list = json.load(f)['data']

In [ ]:
from project_utils import label_str_tolist
sample = file_data_list[80]
label_record = label_str_tolist(sample['labels'])
label_record

['/m/birdc36', '/m/birdc76']

In [ ]:
label_samples_dict = {}
for k, sample in enumerate(file_data_list):
    record_labels = label_str_tolist(sample['labels'])
    for label in record_labels:
        if label in label_samples_dict.keys():
            label_samples_dict[label].append(k)
        else:
            label_samples_dict[label] = [k]

In [ ]:
len(label_samples)

64

In [ ]:
samples_list = [1, 2, 3]
list_len = len(samples_list)
replicate_count_threshold = 10
samples_list = samples_list * int(replicate_count_threshold/ list_len) + samples_list[:(replicate_count_threshold%list_len)]
samples_list

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1]

In [ ]:
for label in label_samples_dict.keys():
    print(f'{label}={len(label_samples_dict[label])}')

/m/birdc41=676
/m/birdc05=1564
/m/birdc00=1210
/m/birdc22=206
/m/birdc01=757
/m/birdc36=1549
/m/birdc50=1154
/m/birdc47=983
/m/birdc69=62
/m/birdc76=186
/m/birdc73=147
/m/birdc19=511
/m/birdc38=40
/m/birdc59=323
/m/birdc14=326
/m/birdc77=23
/m/birdc58=1
/m/birdc64=11
/m/birdc65=55
/m/birdc52=1
/m/birdc74=55
/m/birdc35=1
/m/birdc23=15
/m/birdc03=10
/m/birdc24=12
/m/birdc40=28
/m/birdc56=89
/m/birdc53=3
/m/birdc12=84
/m/birdc44=73
/m/birdc46=42
/m/birdc60=55
/m/birdc62=36
/m/birdc67=33
/m/birdc04=27
/m/birdc09=4
/m/birdc37=4
/m/birdc43=2
/m/birdc61=1
/m/birdc18=16
/m/birdc51=56
/m/birdc26=2
/m/birdc08=4
/m/birdc66=13
/m/birdc45=2
/m/birdc42=2
/m/birdc27=1
/m/birdc34=2
/m/birdc71=2
/m/birdc48=4
/m/birdc28=4
/m/birdc54=1
/m/birdc29=5
/m/birdc55=2
/m/birdc72=4
/m/birdc32=1
/m/birdc20=6
/m/birdc17=2
/m/birdc39=4
/m/birdc57=6
/m/birdc06=9
/m/birdc21=7
/m/birdc75=3
/m/birdc16=8


In [ ]:
minimum_th = 20
replicate_count_threshold = 1000

expand_dict = {}
for label in label_samples_dict.keys():
    samples_list = label_samples_dict[label]
    list_len = len(samples_list)
    # expand only when count is above the minimum filter minimum threshold
    if list_len > minimum_th:
        # expand only when count is below replicate count threshold
        if list_len < replicate_count_threshold:
            # Add (replicate_count_threshold - list_len sample) indexes to sample list
            full_replicates = int((replicate_count_threshold-list_len) / list_len)
            partial_replicate = replicate_count_threshold % list_len
            samples_expand_list = samples_list * full_replicates + samples_list[:partial_replicate]
            expand_dict[label] = samples_expand_list

In [ ]:
replicate_count_threshold

10

In [ ]:
for label in expand_dict.keys():
    print(f'{label}={len(expand_dict[label])}')

/m/birdc41=324
/m/birdc22=794
/m/birdc01=243
/m/birdc47=17
/m/birdc69=938
/m/birdc76=814
/m/birdc73=853
/m/birdc19=489
/m/birdc38=960
/m/birdc59=677
/m/birdc14=674
/m/birdc77=977
/m/birdc65=945
/m/birdc74=945
/m/birdc40=972
/m/birdc56=911
/m/birdc12=916
/m/birdc44=927
/m/birdc46=958
/m/birdc60=945
/m/birdc62=964
/m/birdc67=967
/m/birdc04=973
/m/birdc51=944


In [ ]:
expand_dict

{'/m/birdc41': [0,
  1,
  4,
  5,
  6,
  9,
  13,
  14,
  23,
  24,
  27,
  28,
  29,
  61,
  65,
  69,
  71,
  72,
  74,
  78,
  79,
  81,
  82,
  85,
  120,
  121,
  122,
  123,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  196,
  197,
  363,
  366,
  368,
  369,
  370,
  373,
  552,
  553,
  555,
  556,
  557,
  558,
  559,
  560,
  561,
  563,
  564,
  614,
  615,
  616,
  621,
  626,
  629,
  744,
  745,
  746,
  747,
  748,
  749,
  876,
  877,
  923,
  1140,
  1141,
  1142,
  1143,
  1144,
  1154,
  1155,
  1156,
  1160,
  1161,
  1162,
  1163,
  1164,
  1165,
  1168,
  1169,
  1290,
  1291,
  1292,
  1293,
  1321,
  1324,
  1325,
  1332,
  1333,
  1335,
  1336,
  1338,
  1342,
  1345,
  1346,
  1398,
  1399,
  1400,
  1401,
  1402,
  1405,
  1470,
  1471,
  1472,
  1473,
  1474,
  1475,
  1476,
  1477,
  1478,
  1479,
  1480,
  1481,
  1482,
  1483,
  1484,
  148